<a href="https://colab.research.google.com/github/Hiji1023/dl_network_cla_/blob/main/CH07_03_CNN_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**7-3: Implementation of CNNs**#

##**Code7-3-1: Implementation with Sequential Method**##

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense

N, n_H, n_W, n_C = 4, 28, 28, 3
n_conv_neurons = [10, 20, 30]
n_dense_neurons = [50, 30,10]
k_size, padding = 3, 'same'
pool_size, pool_strides = 2, 2
activation = 'relu'

x = tf.random.normal(shape=(N, n_H, n_W, n_C))
print(x.shape)

# Sequential 장점 layer를 추가하기만 하면 되기에 만들기 간편
# 단점 중간에 Sequential한 commission이 아니라 다른 식의 텐서 연산은 안됨
model= Sequential()
model.add(Conv2D(filters=n_conv_neurons[0], kernel_size = k_size, padding = padding, activation = activation))
model.add(MaxPooling2D(pool_size = pool_size, strides = pool_strides))
model.add(Conv2D(filters=n_conv_neurons[1], kernel_size = k_size, padding = padding, activation = activation))
model.add(MaxPooling2D(pool_size = pool_size, strides = pool_strides))
model.add(Conv2D(filters=n_conv_neurons[2], kernel_size = k_size, padding = padding, activation = activation))
model.add(MaxPooling2D(pool_size = pool_size, strides = pool_strides))
model.add(Flatten())
model.add(Dense(units=n_dense_neurons[0], activation = activation))
model.add(Dense(units=n_dense_neurons[1], activation = activation))
model.add(Dense(units=n_dense_neurons[2], activation = 'softmax'))

predictions = model(x)
print(predictions.shape)


(4, 28, 28, 3)
(4, 10)


In [ ]:
model= Sequential()
for n_conv_neuron in n_conv_neurons:
  model.add(Conv2D(filters=n_conv_neuron, kernel_size = k_size, padding = padding, activation = activation))
  model.add(MaxPooling2D(pool_size = pool_size, strides = pool_strides))
model.add(Flatten())
for n_dense_neuron in n_dense_neurons:
  model.add(Dense(units=n_dense_neuron, activation = activation))
model.add(Dense(units=n_dense_neurons[-1], activation = 'softmax'))


##**Code7-3-2: Implementation with Model Sub-classing**##

In [ ]:
import tensorflow as tf

from tensorflow.keras.models import Model

from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense

class TestCNN(Model):
  def __init__(self):
    super(TestCNN, self).__init__()

    self.conv1 = Conv2D(filters=n_conv_neurons[0], kernel_size = k_size, padding = padding, activation = activation)
    self.conv1_pool = MaxPooling2D(pool_size = pool_size, strides = pool_strides)
    self.conv2 = Conv2D(filters=n_conv_neurons[1], kernel_size = k_size, padding = padding, activation = activation)
    self.conv2_pool = MaxPooling2D(pool_size = pool_size, strides = pool_strides)
    self.conv3 = Conv2D(filters=n_conv_neurons[2], kernel_size = k_size, padding = padding, activation = activation)
    self.conv3_pool = MaxPooling2D(pool_size = pool_size, strides = pool_strides)
    self.flatten = Flatten()

    self.dense1 = Dense(units=n_dense_neurons[0], activation = activation)
    self.dense2 = Dense(units=n_dense_neurons[1], activation = activation)
    self.dense3 = Dense(units=n_dense_neurons[2], activation = 'softmax')

  def call(self, x):
    print(x.shape)
    x = self.conv1(x)
    print(x.shape)
    x = self.conv1_pool(x)
    print(x.shape)

    x = self.conv2(x)
    print(x.shape)
    x = self.conv2_pool(x)
    print(x.shape)
    
    x = self.conv3(x)
    print(x.shape)
    x = self.conv3_pool(x)
    print(x.shape)

    x = self.flatten(x)
    print(x.shape)

    x = self.dense1(x)
    print(x.shape)
    x = self.dense2(x)
    print(x.shape)
    x = self.dense3(x)
    print(x.shape)
    return x

N, n_H, n_W, n_C = 4, 28, 28, 3
n_conv_neurons = [10, 20, 30]
n_dense_neurons = [50, 30,10]
k_size, padding = 3, 'same'
pool_size, pool_strides = 2, 2
activation = 'relu'

x = tf.random.normal(shape=(N, n_H, n_W, n_C))

model = TestCNN()
y = model(x)
print(y.shape)

(4, 28, 28, 3)
(4, 28, 28, 10)
(4, 14, 14, 10)
(4, 14, 14, 20)
(4, 7, 7, 20)
(4, 7, 7, 30)
(4, 3, 3, 30)
(4, 270)
(4, 50)
(4, 30)
(4, 10)
(4, 10)


##**Code7-3-3: Implementation with Sequential + Layer Sub-classing**##

In [ ]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense

# Layer Sub-classing
# Sequential로 만들면 layer갯수만큼 다 작성해야함, layer가 많아질수록 코드가 간단해짐
class MyConv(Layer):
  def __init__(self, n_neuron):
    super(MyConv, self).__init__()

    self.conv = Conv2D(filters=n_neuron, kernel_size = k_size, padding = padding, activation = activation)
    self.conv_pool = MaxPooling2D(pool_size = pool_size, strides = pool_strides)

  def call(self, x):
    x = self.conv(x)
    x = self.conv_pool(x)
    return x

model = Sequential()
# n_conv_neurons[] -> MyConv의 n_neuron자리에 들어감
model.add(MyConv(n_conv_neurons[0]))
model.add(MyConv(n_conv_neurons[1]))
model.add(MyConv(n_conv_neurons[2]))
model.add(Flatten())

model.add(Dense(units=n_dense_neurons[0], activation = activation))
model.add(Dense(units=n_dense_neurons[1], activation = activation))
model.add(Dense(units=n_dense_neurons[2], activation = 'softmax'))

##**Code7-3-4: Implementation with Model and Layer Sub-classing**##

In [ ]:
import tensorflow as tf

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense

# Layer Sub-classing
# Sequential로 만들면 layer갯수만큼 다 작성해야함, layer가 많아질수록 코드가 간단해짐
class MyConv(Layer):
  def __init__(self, n_neuron):
    super(MyConv, self).__init__()

    self.conv = Conv2D(filters=n_neuron, kernel_size = k_size, padding = padding, activation = activation)
    self.conv_pool = MaxPooling2D(pool_size = pool_size, strides = pool_strides)

  def call(self, x):
    x = self.conv(x)
    x = self.conv_pool(x)
    return x

class TestCNN(Model):
  def __init__(self):
    super(TestCNN, self).__init__()

    self.conv1 = MyConv(n_conv_neurons[0])
    self.conv2 = MyConv(n_conv_neurons[1])
    self.conv3 = MyConv(n_conv_neurons[2])
    self.flatten = Flatten()

    self.dense1 = Dense(units=n_dense_neurons[0], activation = activation)
    self.dense2 = Dense(units=n_dense_neurons[1], activation = activation)
    self.dense3 = Dense(units=n_dense_neurons[2], activation = 'softmax')

  def call(self, x):
    x = self.conv1(x)
    x = self.conv2(x)
    x = self.conv3(x)
    x = self.flatten(x)

    x = self.dense1(x)
    x = self.dense2(x)
    x = self.dense3(x)
    return x


In [ ]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense

class MyConv(Layer):
  def __init__(self, n_neuron):
    super(MyConv, self).__init__()

    self.conv = Conv2D(filters=n_neuron, kernel_size = k_size, padding = padding, activation = activation)
    self.conv_pool = MaxPooling2D(pool_size = pool_size, strides = pool_strides)

  def call(self, x):
    x = self.conv(x)
    x = self.conv_pool(x)
    return x

class TestCNN(Model):
  def __init__(self):
    super(TestCNN, self).__init__()
    self.fe = Sequential()

    self.fe.add(MyConv(n_conv_neurons[0]))
    self.fe.add(MyConv(n_conv_neurons[1]))
    self.fe.add(MyConv(n_conv_neurons[2]))
    self.fe.add(Flatten())

    self.classifier = Sequential()

    self.classifier.add(Dense(units=n_dense_neurons[0], activation = activation))
    self.classifier.add(Dense(units=n_dense_neurons[1], activation = activation))
    self.classifier.add(Dense(units=n_dense_neurons[2], activation = 'softmax'))

  def call(self, x):
    # x = self.conv1(x)
    # x = self.conv2(x)
    # x = self.conv3(x)
    # x = self.flatten(x)
    x = self.fe()

    # x = self.dense1(x)
    # x = self.dense2(x)
    # x = self.dense3(x)
    x = self.classifier()
    return x